In [1]:
import sys
import os
import pandas as pd
from langchain_openai import ChatOpenAI


# Add project root to sys.path in Jupyter or interactive session
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../../")))

from config.path_config import DATA_DIR, PROJECT_DIR, DEPENDENCIES_DIR, CONFIG_DIR
from src.helper_functions.data_pre_processing.data_upload_processor.processor import *
from src.helper_functions.database_upload.postgres_uploader import *

from src.helper_functions.query_mapping.query_mapping_functions import *
from src.helper_functions.tools.tools import *
from src.helper_functions.data_pre_processing.data_upload_processor.processor import *
from src.helper_functions.data_pre_processing.data_filtering.filter_dataframe import *

from dotenv import load_dotenv

In [2]:
llm_ranking_prompt_path = CONFIG_DIR+"\\llm_ranking_prompt.yaml"
llm_ranking_prompt_path

'c:\\Users\\73315\\OneDrive - Bain\\Documents\\CL_Chatbot_V_01\\config\\llm_ranking_prompt.yaml'

In [3]:
os.getcwd()

'c:\\Users\\73315\\OneDrive - Bain\\Documents\\CL_Chatbot_V_01\\src\\helper_functions\\tools'

In [4]:
env_path = os.path.join(DEPENDENCIES_DIR, ".env")
load_dotenv(env_path)
api_key = os.getenv("OPENAI_API_KEY")

In [5]:
model = ChatOpenAI(model='gpt-4o',api_key=api_key,temperature=0)

C:\Users\73315\AppData\Local\Temp\ipykernel_25040\2881712433.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(model='gpt-4o',api_key=api_key,temperature=0)


In [6]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

In [7]:
src_path = os.path.abspath(os.path.join(os.getcwd(), "src"))
if src_path not in sys.path:
    sys.path.append(src_path)

In [8]:
os.getcwd()

'c:\\Users\\73315\\OneDrive - Bain\\Documents\\CL_Chatbot_V_01\\src\\helper_functions\\tools'

In [9]:
yaml_file_path = "testing_mapping.yaml"
print(yaml_file_path)

testing_mapping.yaml


In [10]:
raw_data = pd.read_excel(os.path.join(DATA_DIR, "wave_10_responses.xlsx"))
data_map = pd.read_excel(os.path.join(DATA_DIR, "wave_10_question_mapping.xlsx"))

In [11]:
env_path = os.path.join(DEPENDENCIES_DIR, ".env")
load_dotenv(env_path)
api_key = os.getenv("OPENAI_API_KEY")

In [12]:
from langchain_openai import OpenAI

In [13]:
data_dict = run_question_data_pipeline(data_map, raw_data, api_key,'wave_10')


🚨 DEBUG: Inside process_question_guide
Shape of data_map: (786, 2)
Columns in data_map: ['Q80', 'Where do you live?']
First 5 rows:
   Q80 Where do you live?
0   1      United States
1   2     United Kingdom
2   3              Italy
3   4             France
4   5            Germany


c:\Users\73315\OneDrive - Bain\Documents\CL_Chatbot_V_01\src\helper_functions\data_pre_processing\data_upload_processor\question_embedder.py:66: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(api_key=api_key)


In [14]:
question_dict_with_embedding = data_dict['question_dict']
question_dict_with_embedding

[{'question_code': 'Q10',
  'question_text': 'What is your current employment status?',
  'question_with_answers': 'What is your current employment status? Full-time employed in one job (working >30 hours per week)|| Full-time employed in more than one job (working >30 hours per week)|| Part-time employed in one job (working <30 hours per week)|| Part-time employed in more than one job (working <30 hours per week)|| Contractor, freelance, temporary, self-employed|| Business owner|| Full-time student and part-time employed|| Full-time student and not employed|| Stay-at-home parent or caregiver|| Unemployed|| Retired|| None of the above',
  'answer_options': 'Full-time employed in one job (working >30 hours per week)|| Full-time employed in more than one job (working >30 hours per week)|| Part-time employed in one job (working <30 hours per week)|| Part-time employed in more than one job (working <30 hours per week)|| Contractor, freelance, temporary, self-employed|| Business owner|| Ful

In [15]:
type_sub_type_df = data_dict['type_subtype']

In [16]:
prompt_dict = read_yaml(llm_ranking_prompt_path)
prompt_dict

{'system_prompt': "You are an analyst, working for Bain Consulting's consumer practice COE.",
 'user_prompt': 'You allow a user to ask a question and then, from a list of questions from a consumer survey, you identify the questions that answer\ntheir query most accurately. The query is: {query}\nand the list of questions is this:\n{formatted_questions}\n**************************\n\nPlease rank these questions in order of how accurately the questions answer the query.\nI want you to rank all the questions and then return a list of the question numbers in a decreasing order of relevancy.\nDo not repeat the same question in the list.\n\nONLY RETURN THE LIST AND NOTHING ELSE. For example: [2,3,1]'}

In [17]:
prompt_for_llm_sorting = promt_to_sort_llm(prompt_dict)
prompt_for_llm_sorting

ChatPromptTemplate(input_variables=['formatted_questions', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are an analyst, working for Bain Consulting's consumer practice COE."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['formatted_questions', 'query'], input_types={}, partial_variables={}, template='You allow a user to ask a question and then, from a list of questions from a consumer survey, you identify the questions that answer\ntheir query most accurately. The query is: {query}\nand the list of questions is this:\n{formatted_questions}\n**************************\n\nPlease rank these questions in order of how accurately the questions answer the query.\nI want you to rank all the questions and then return a list of the question numbers in a decreasing order of relevancy.\nDo not repeat the same question in 

In [18]:
mapped_data = data_dict['mapped_data']

In [19]:
default_filters_path = CONFIG_DIR+'\\default_filters\\wave_10_default_filters.yaml'
default_filters_path

'c:\\Users\\73315\\OneDrive - Bain\\Documents\\CL_Chatbot_V_01\\config\\default_filters\\wave_10_default_filters.yaml'

In [20]:
default_filters = read_yaml(default_filters_path)
default_filters

{'Age': {'mapping': {'Baby Boomers': ['65', '200'],
   'Gen X': ['45', '64'],
   'Gen Z': ['0', '26'],
   'Millennials': ['27', '44']},
  'filter_name': 'Generation Filter',
  'survey_question': 'Q4',
  'filter_type': 'categorize'},
 'Alcohol-Consumption': {'mapping': {'Alcohol Averse': ['I’m drinking less alcohol than I used to',
    "I don't consume alcohol"],
   'Alcohol Consumer': ['I’m drinking about the same amount of alcohol',
    'I’m drinking more alcohol than I used to']},
  'filter_name': 'Alcohol Consumption Filter',
  'survey_question': 'Q55',
  'filter_type': 'categorize'},
 'Political-Alignment': {'mapping': {'No Strong Alignment': ['1 - Not at all aligned',
    '2',
    '4',
    '3 - Neutral'],
   'Strongly Aligned': ['5 - Strongly aligned']},
  'filter_name': 'Political Alignment Filter',
  'survey_question': 'Q59',
  'filter_type': 'categorize'},
 'Geographical-Area': {'filter_name': 'Area Filter',
  'survey_question': 'Q6',
  'filter_type': 'recreate'},
 'Country': {

In [21]:
final_processed_raw_df_filter = add_filtering_columns(mapped_data,default_filters)
final_processed_raw_df_filter

,Respondent,Status,Term_reason,Start_time_GMT,End_time_GMT,Panel,country,first_param,second_param,income_class,...,Q79__user_input,Generation Filter,Alcohol Consumption Filter,Political Alignment Filter,Area Filter,Country Filter,Gender Filter,Income Filter,Political Affinity Filter,Region Filter
0,5AfmucpVJF63JuU32MC8Bg**,Qualified,NaN,2024-10-01 14:13:15.565,2024-10-01 14:22:36.490,Dynata - France,4,Charente-Maritime,Nouvelle-Aquitaine,3,...,NaN,Gen X,Alcohol Consumer,No Strong Alignment,Village,France,Male / man,High,Center,NaN
1,5AfmucpVJF6BwCDVcf_cQw**,Qualified,NaN,2024-10-01 14:14:50.335,2024-10-01 14:24:07.254,Dynata - Italy,3,Ravenna,Emilia-Romagna,2,...,NaN,Gen X,Alcohol Consumer,No Strong Alignment,Suburb,Italy,Female / woman,Mid,Center right,NaN
2,5AfmucpVJF5qH3Ly_L-u1Q**,Qualified,NaN,2024-10-01 14:16:36.935,2024-10-01 14:24:34.457,Dyanta - Spain,7,Madrid,Madrid,3,...,NaN,Millennials,Alcohol Consumer,No Strong Alignment,Suburb,Spain,Male / man,High,Right,NaN
3,5AfmucpVJF4WooWl3om3mw**,Qualified,NaN,2024-10-01 14:15:35.866,2024-10-01 14:24:42.186,Dynata - Poland,6,Małopolskie,Powiat proszowicki,1,...,NaN,Millennials,Alcohol Averse,NaN,Village,Poland,Female / woman,Low,None,NaN
4,5AfmucpVJF6P1Eb2aefL1A**,Qualified,NaN,2024-10-01 14:16:47.263,2024-10-01 14:25:02.217,Dynata - UK,2,E30000254,E12000008,1,...,NaN,Gen X,Alcohol Averse,NaN,Suburb,United Kingdom,Female / woman,Low,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,xjUBnUXDVJ1oT0NhUlBY6g**,Qualified,NaN,2024-10-11 14:40:46.093,2024-10-11 15:00:57.319,Dynata - Poland,6,Lubelskie,"Chełm County, Chełm",3,...,NaN,Gen X,Alcohol Averse,No Strong Alignment,City (not center),Poland,Male / man,High,Center,NaN
10496,xjUBnUXDVJ1HREX-hkINKQ**,Qualified,NaN,2024-10-11 14:27:23.980,2024-10-11 15:02:17.112,Dynata - Poland,6,Mazowieckie,Warszawa,1,...,NaN,Gen X,Alcohol Averse,NaN,City center,Poland,Female / woman,Low,None,NaN
10497,xjUBnUXDVJ0wOtvGv1TYkQ**,Qualified,NaN,2024-10-11 14:43:35.136,2024-10-11 15:04:08.265,Dynata - Poland,6,Śląskie,Katowice,1,...,NaN,Gen X,Alcohol Averse,No Strong Alignment,City center,Poland,Female / woman,Low,Center left,NaN
10498,xjUBnUXDVJ0lCWdkspmwQQ**,Qualified,NaN,2024-10-11 14:26:29.472,2024-10-11 15:05:30.095,Dynata - Poland,6,Wielkopolskie,Poznań,2,...,NaN,Millennials,Alcohol Averse,NaN,City center,Poland,Female / woman,Mid,None,NaN


In [22]:
agents_prompt_file_name = CONFIG_DIR+"\\agent_prompt.yaml"
agents_prompt_file_name

'c:\\Users\\73315\\OneDrive - Bain\\Documents\\CL_Chatbot_V_01\\config\\agent_prompt.yaml'

In [23]:
agents_prompt = read_yaml(agents_prompt_file_name)
agents_prompt

{'custom_agent_prefix': '""" You are a Survey Data Expert who helps users analyze consumer survey results using available tools.\nGuidelines: - Always call the appropriate tool to generate a **pivot table**. - In your **Final Answer**, include the **pivot table** output directly. Do **not** just summarize it — it must be part of the response. - Return the pivot table exactly as provided (in markdown format if available). - After the table, include 2-3 pointers of clear, data-driven insights backed by numbers in percentages (e.g., "Gen Z reported the highest interest in second-hand shopping."). - Do not paraphrase or interpret the table before showing it — show the table first, then write insights. - Do not skip the table under any circumstances, even if the insight is obvious. - If the table is empty, apologize and ask the user to rephrase or check their filters. """',
 'markdown': {'system_prompt': 'You are a helpful assistant.',
  'human_prompt_1': 'provide answer for the query: {que

In [24]:
Filters = generate_filter_value_dict(final_processed_raw_df_filter,default_filters)
Filters

{'Generation Filter': 'Baby Boomers, Gen X, Gen Z, Millennials',
 'Alcohol Consumption Filter': 'Alcohol Averse, Alcohol Consumer',
 'Political Alignment Filter': 'No Strong Alignment, Strongly Aligned',
 'Area Filter': 'City (not center), City center, Other (please specify), Suburb, Town, Village',
 'Country Filter': 'France, Germany, Italy, Poland, Spain, United Kingdom, United States',
 'Gender Filter': 'Female / woman, Gender-fluid / Non-binary, Male / man, None of these options describe me, Prefer not to answer\xa0',
 'Income Filter': 'High, Low, Mid, No Response',
 'Political Affinity Filter': 'Center, Center left, Center right, Far-Left, Far-Right, Left, None, Other, Right',
 'Region Filter': 'East North Central, East South Central, Middle Atlantic, Mountain, New England, Pacific, South Atlantic, West North Central, West South Central'}

In [25]:
final_processed_raw_df_filter

,Respondent,Status,Term_reason,Start_time_GMT,End_time_GMT,Panel,country,first_param,second_param,income_class,...,Q79__user_input,Generation Filter,Alcohol Consumption Filter,Political Alignment Filter,Area Filter,Country Filter,Gender Filter,Income Filter,Political Affinity Filter,Region Filter
0,5AfmucpVJF63JuU32MC8Bg**,Qualified,NaN,2024-10-01 14:13:15.565,2024-10-01 14:22:36.490,Dynata - France,4,Charente-Maritime,Nouvelle-Aquitaine,3,...,NaN,Gen X,Alcohol Consumer,No Strong Alignment,Village,France,Male / man,High,Center,NaN
1,5AfmucpVJF6BwCDVcf_cQw**,Qualified,NaN,2024-10-01 14:14:50.335,2024-10-01 14:24:07.254,Dynata - Italy,3,Ravenna,Emilia-Romagna,2,...,NaN,Gen X,Alcohol Consumer,No Strong Alignment,Suburb,Italy,Female / woman,Mid,Center right,NaN
2,5AfmucpVJF5qH3Ly_L-u1Q**,Qualified,NaN,2024-10-01 14:16:36.935,2024-10-01 14:24:34.457,Dyanta - Spain,7,Madrid,Madrid,3,...,NaN,Millennials,Alcohol Consumer,No Strong Alignment,Suburb,Spain,Male / man,High,Right,NaN
3,5AfmucpVJF4WooWl3om3mw**,Qualified,NaN,2024-10-01 14:15:35.866,2024-10-01 14:24:42.186,Dynata - Poland,6,Małopolskie,Powiat proszowicki,1,...,NaN,Millennials,Alcohol Averse,NaN,Village,Poland,Female / woman,Low,None,NaN
4,5AfmucpVJF6P1Eb2aefL1A**,Qualified,NaN,2024-10-01 14:16:47.263,2024-10-01 14:25:02.217,Dynata - UK,2,E30000254,E12000008,1,...,NaN,Gen X,Alcohol Averse,NaN,Suburb,United Kingdom,Female / woman,Low,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,xjUBnUXDVJ1oT0NhUlBY6g**,Qualified,NaN,2024-10-11 14:40:46.093,2024-10-11 15:00:57.319,Dynata - Poland,6,Lubelskie,"Chełm County, Chełm",3,...,NaN,Gen X,Alcohol Averse,No Strong Alignment,City (not center),Poland,Male / man,High,Center,NaN
10496,xjUBnUXDVJ1HREX-hkINKQ**,Qualified,NaN,2024-10-11 14:27:23.980,2024-10-11 15:02:17.112,Dynata - Poland,6,Mazowieckie,Warszawa,1,...,NaN,Gen X,Alcohol Averse,NaN,City center,Poland,Female / woman,Low,None,NaN
10497,xjUBnUXDVJ0wOtvGv1TYkQ**,Qualified,NaN,2024-10-11 14:43:35.136,2024-10-11 15:04:08.265,Dynata - Poland,6,Śląskie,Katowice,1,...,NaN,Gen X,Alcohol Averse,No Strong Alignment,City center,Poland,Female / woman,Low,Center left,NaN
10498,xjUBnUXDVJ0lCWdkspmwQQ**,Qualified,NaN,2024-10-11 14:26:29.472,2024-10-11 15:05:30.095,Dynata - Poland,6,Wielkopolskie,Poznań,2,...,NaN,Millennials,Alcohol Averse,NaN,City center,Poland,Female / woman,Mid,None,NaN


In [90]:

import sys
import os

# Add project root to sys.path in Jupyter or interactive session
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../../../")))

from src.helper_functions.query_mapping.query_mapping_functions import *
from src.helper_functions.CL_Cuts.cuts import *
import json

####################################################################################################################

def extract_filters(query: str, filter_dict: dict) -> dict:
    """
    Returns a dictionary:
        {
           "Generation Filter": [matched_values...],
           "Political Consumption Filter": [matched_values...],
           ...
        }
    where each key corresponds to a filter category, and the value is
    a list of all recognized filter values found in `query`.
    """
    FILTERS = {}
    for key, value in filter_dict.items():
        # Split each string by commas and strip whitespace
        FILTERS[key] = [item.strip() for item in value.split(',')]
    filter_dict = FILTERS
    
    # Convert query to lowercase for case-insensitive matching
    query_lower = query.lower()

    # Prepare an output dictionary with empty lists
    detected_filters = {f: [] for f in filter_dict}

    for filter_name, filter_values in filter_dict.items():
        for val in filter_values:
            # Simple substring check
            # Lowercase the filter value for matching
            val_lower = val.lower()

            # If the filter value is multiple words or has punctuation,
            # you might do a more careful match. Here we keep it simple:
            if val_lower in query_lower:
                detected_filters[filter_name].append(val)

    # Remove filter categories that had no matches
    detected_filters = {k: v for k, v in detected_filters.items() if v}
    return detected_filters



def extract_demographics(query: str, filter_dict: dict) -> list:
    """
    Given a user query (string) and the FILTERS dictionary,
    return a list of all filter "demographics" (keys) 
    found in the query via case-insensitive substring match.
    """
    FILTERS = {}
    for key, value in filter_dict.items():
        # Split each string by commas and strip whitespace
        FILTERS[key] = [item.strip() for item in value.split(',')]
    filter_dict = FILTERS
    
    query_lower = query.lower()
    demographics_found = []
    
    for demographic_key in filter_dict.keys():
        demographic_key_modified = demographic_key.replace(" Filter","")
        if demographic_key_modified.lower() in query_lower:
            demographics_found.append(demographic_key)
    
    return demographics_found



###########################################################################################################################


def make_get_question_info_tool(question_dict,prompt_for_sorting,model,type_subtype_df,api_key):
    def _get_best_question_info_tool(query: str) -> dict:
        return get_best_question_info(
            query=query,
            question_dict_with_embedding=question_dict,
            prompt_for_llm_sorting=prompt_for_sorting,
            model=model,
            type_sub_type_df=type_subtype_df,
            api_key = api_key
        )
    return _get_best_question_info_tool



def make_multi_select_pivot_tool(df,filters):
    def _multi_select_pivot_tool(tool_input: str) -> str:
        print(tool_input)
        try:
            # Handles both actual dict input and stringified JSON input
            tool_args =ast.literal_eval(tool_input) #tool_input if isinstance(tool_input, dict) else json.loads(tool_input)
            print("\n",tool_args)

            query = tool_args["query"]
            question_id = tool_args["question_id"]

            demographics = extract_demographics(query,filters)
            if not isinstance(demographics, list):
                demographics = [demographics]

            print(demographics)

        except Exception as e:
            return f"Tool failed to parse input: {e}"

        try:
            local_df = df.copy()

            # Apply any filters detected from the user query
            extra_filters = extract_filters(query, filters)
            print(extra_filters)
            filtered_dict = {k: v for k, v in extra_filters.items() if k not in demographics}
            local_df = filter_dataframe(local_df,filtered_dict)

            # Proceed with pivot
            question_cols = [col for col in local_df.columns if col.startswith(question_id)]
            if not question_cols:
                raise ValueError("No columns found for the question ID")
            
            

            melted = local_df[question_cols + demographics].melt(
                id_vars=demographics,
                var_name="Question_Option",
                value_name="Selected"
            )
            filtered = melted[melted["Selected"] == 1]
            grouped = filtered.groupby(["Question_Option"] + demographics).size().reset_index(name="Count")
            pivot_table = grouped.pivot_table(index="Question_Option", columns=demographics, values="Count", fill_value=0)

            pivot_table.loc["Total"] = pivot_table.sum()
            return pivot_table.to_markdown()
        except Exception as e:
            return f"Pivot generation failed: {e}"
    return _multi_select_pivot_tool




def make_pivot_count_tool(df,filters):
    def _pivot_count_tool(tool_input: str) -> str:
        try:
            tool_args = tool_input if isinstance(tool_input, dict) else ast.literal_eval(tool_input)
            query = tool_args["query"]
            question_id = tool_args["question_id"]


            demographics = extract_demographics(query=query,filter_dict=filters)

            if not isinstance(demographics, list):
                demographics = [demographics]

                

        except Exception as e:
            return f"Tool failed to parse input: {e}"

        try:
            local_df = df.copy()

            # Extra filters from input (e.g., income=Low)
            extra_filters = extract_filters(query, filters)
            print(extra_filters)
            filtered_dict = {k: v for k, v in extra_filters.items() if k not in demographics}
            local_df = filter_dataframe(local_df,filtered_dict)

            # Validate that the column exists
            if question_id not in local_df.columns:
                return f"Column '{question_id}' not found."

            # Build pivot
            pivot = local_df.pivot_table(index=question_id, columns=demographics, aggfunc="size", fill_value=0)
            pivot.loc["Total"] = pivot.sum()
            return pivot.to_markdown()
        except Exception as e:
            return f"Pivot generation failed: {e}"
    return _pivot_count_tool

In [91]:
get_best_question_info_tool = make_get_question_info_tool(question_dict_with_embedding,prompt_for_llm_sorting,model = model, type_subtype_df=type_sub_type_df,api_key = api_key)
pivot_count_tool = make_pivot_count_tool(final_processed_raw_df_filter,Filters)
multi_select_pivot_tool = make_multi_select_pivot_tool(final_processed_raw_df_filter,Filters)

In [ ]:
from langchain.tools import Tool

tools = [
    Tool(
        name="GetBestQuestionInfo",
        func=get_best_question_info_tool,
        description="Given a user query, returns the best matching survey question and its type/subtype."
    ),
    Tool(
        name="MultiSelectPivot",
        func=multi_select_pivot_tool,
        description=(
    "Generate pivot table for a multi-select question. "
    "Input can be a Python dictionary with the following keys:\n"
    "- 'query': the full user question\n"
    "- 'question_id': the question code \n"
)
    ),
    Tool(
        name="SingleSelectPivot",
        func=pivot_count_tool,
        description=(
    "Generate pivot table for a single-select question. "
    "Input can be a Python dictionary with the following keys:\n"
    "- 'query': the full user question\n"
    "- 'question_id': the question code \n"
)
    )
]

In [111]:
from langchain.agents import initialize_agent
from langchain.agents.agent_types import AgentType

custom_agent = initialize_agent(
    tools=tools,
    llm=model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    handle_parsing_errors=True,
    agent_kwargs = {
    "prefix": agents_prompt['custom_agent_prefix']
}
)


In [112]:
query = "Which generation is most likely to cut salon visits first?"

In [113]:
from io import StringIO

In [114]:
def markdown_table_to_df(md_str):
    """
    Convert a Markdown table (GitHub-style) into a pandas DataFrame.
    
    Args:
        md_str (str): The Markdown table as a string.
    
    Returns:
        pd.DataFrame: DataFrame containing the table data.
    """
    # Split the input into lines, stripping away extra surrounding whitespace and '|'.
    lines = md_str.strip().split('\n')
    lines = [line.strip().strip('|') for line in lines if line.strip()]

    # Build a list of lines excluding the 'separator' line (---|--- etc.)
    clean_lines = []
    for line in lines:
        # Remove all spaces for the check
        check = line.replace(' ', '')
        # If all characters are '-', ':', or '|', consider this a separator line and skip it
        if all(ch in '-:|' for ch in check):
            continue
        clean_lines.append(line)

    # Join the cleaned lines into a single string
    text_for_csv = '\n'.join(clean_lines)

    # Parse with pandas.read_csv, using '|' as the delimiter
    df = pd.read_csv(StringIO(text_for_csv), sep='|', engine='python')

    # Clean up column names and strip any extra whitespace in data cells
    df.columns = [c.strip() for c in df.columns]
    for col in df.columns:
        if df[col].dtype == 'object':
            df[col] = df[col].str.strip()
    df = df.dropna()
    return df

In [115]:
def custom_agent_layer(query, custom_agent):
    response = custom_agent.run(query)
    print("Response",response)
    
    try:
        final_response = response.split("Insights:")
        output_dict = {
            "Data": markdown_table_to_df(final_response[0]),
            "Insights": final_response[1].strip()
        }
    except Exception as e:
        output_dict = {
            "Data": markdown_table_to_df(response),  # still try parsing the table
            "Insights": f" Unable to extract 'Insights:' section. Error: {e}\n\nFull response:\n{response}"
        }
    
    return output_dict

In [116]:
custom_agent_output = custom_agent_layer(query,custom_agent)



> Entering new AgentExecutor chain...
I need to identify the best matching survey question related to generations and their likelihood of cutting salon visits. 

Action: GetBestQuestionInfo
Action Input: "Which generation is most likely to cut salon visits first?"['Q36', 'Q34', 'Q37']

Observation: {'question_id': 'Q36', 'question_text': 'What actions have you taken to reduce your spending on Beauty treatments at the salon (e.g., haircuts, nail salon)? Please select up to 3 answers.\xa0', 'type': 'Multiple choice', 'subtype': 'multi-select', 'full_row':     Question_no.                                           Question  \
274          Q36  What actions have you taken to reduce your spe...   

                Type      Sub-type  
274  Multiple choice  multi-select  }
Thought:The best matching survey question is a multi-select question about actions taken to reduce spending on beauty treatments at the salon. I need to generate a pivot table for this question to determine which generat

In [118]:
a = ast.literal_eval('{"query": "Which generation is most likely to cut salon visits?", "question_id": "Q36"}')


In [88]:
a

{'query': 'Which generation is most likely to cut salon visits?',
 'question_id': 'Q36'}

In [89]:
a.keys()

dict_keys(['query', 'question_id'])